In [ ]:
import json
import time

from kafka import KafkaProducer


def json_serializer(data):
    return json.dumps(data).encode("utf-8")


server = "localhost:9092"

producer = KafkaProducer(bootstrap_servers=[server], value_serializer=json_serializer)

producer.bootstrap_connected()

In [ ]:
t0 = time.time()

topic_name = "test-topic"

for i in range(10):
    message = {"number": i}
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
    time.sleep(0.05)

t1 = time.time()
print(f"sending message took {(t1 - t0):.2f} seconds")

t3 = time.time()
producer.flush()
t4 = time.time()
print(f"flushing took {(t4 - t3):.2f} seconds")

### Sending Green trips data


In [ ]:
import pandas as pd

parse_dates = ["lpep_pickup_datetime", "lpep_dropoff_datetime"]
columns = [
    "lpep_pickup_datetime",
    "lpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "passenger_count",
    "trip_distance",
    "tip_amount",
]
df_green = pd.read_csv(
    "green_tripdata_2019-10.csv.gz",
    compression="gzip",
    usecols=columns,
)

In [ ]:
df_green.head()

In [ ]:
topic_name = "green-trips"
t0 = time.time()
for row in df_green.itertuples():
    row_dict = {col: getattr(row, col) for col in row._fields}
    message = json.dumps(row_dict)
    producer.send(topic_name, value=message)
    print(f"Sent: {message}")
t1 = time.time()
print(f"sending message took {(t1 - t0):.2f} seconds")